In [77]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, split, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import DateType, StringType, IntegerType
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, IsotonicRegression, RandomForestRegressor
from pyspark.ml.linalg import Vectors, DenseVector, VectorUDT
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [7]:
from datetime import datetime

In [2]:
sc = SparkContext()
sqlc = SparkSession(sc)

In [13]:
change_to_month_func = udf(lambda record: int(datetime.strftime(datetime.strptime(record, '%d/%m/%Y'), '%Y%m')), IntegerType())
change_to_date_func = udf(lambda record: datetime.strptime(str(record), '%Y%m'), DateType())
to_vector = udf(lambda record: Vectors.dense(record), VectorUDT())
to_vectors = udf(lambda col_a, col_b: Vectors.sparse(col_a, col_b))

In [14]:
reading = sqlc.read.csv('Admissions 2015-16.csv', header=True)
reading_mod = reading.withColumn('Date', change_to_month_func(col('Removal Date'))).drop('Removal Date'
                                        ).withColumnRenamed('count(Removal Date)', 'patients_removed')

In [38]:
reading_mod.show(5)

+--------+---------------+-----------+-------------+--------------+--------+------------+------+
|Hospital|      Specialty|  Procedure|       Doctor|Patient Number|Priority|Waiting Days|  Date|
+--------+---------------+-----------+-------------+--------------+--------+------------+------+
|     Mel|General Surgery|Lap Banding|Joseph Miller|     111365825|       C|        3396|201508|
|     Mel|General Surgery|Lap Banding|Joseph Miller|     109970143|       C|        3356|201507|
|     Mel|General Surgery|Lap Banding|Joseph Miller|     106770523|       C|        3244|201508|
|     Mel|General Surgery|Lap Banding|Joseph Miller|     111176864|       C|        3229|201508|
|     Mel|General Surgery|Lap Banding|Joseph Miller|     107085813|       C|        3190|201508|
+--------+---------------+-----------+-------------+--------------+--------+------------+------+
only showing top 5 rows



In [15]:
grouped = reading_mod.groupby('Date').agg({'Date': 'count'})
grouped_with_date = grouped.withColumn('Date', change_to_date_func(col('Date')))
window_row = Window().orderBy('Date')
grouped_new = grouped_with_date.withColumn('id', row_number().over(window_row))

In [16]:
grouped_new_1 = grouped_new.withColumn('id', to_vector(col('id')))
testing_df = grouped_new_1.where(col('Date') > datetime(2016,4,2))
training_df = grouped_new_1.filter(col('Date') < datetime(2016,6,2))

In [ ]:
lr = LinearRegression(maxIter=100, regParam=0.01, elasticNetParam = 1.0, labelCol="count(Date)", featuresCol="id")
model_lr = lr.fit(training_df.select('count(Date)', 'id'))
model_lr.coefficients
transformed_lr = model_lr.transform(testing_df)

In [18]:
dt = DecisionTreeRegressor(maxDepth=3, labelCol="count(Date)", featuresCol="id")
model_dt = dt.fit(training_df.select('count(Date)', 'id'))
transformed_dt = model_dt.transform(testing_df)

In [81]:
dt = DecisionTreeRegressor()
params = ParamGridBuilder().baseOn({dt.labelCol: "count(Date)"}).baseOn({dt.featuresCol: "id"}).addGrid(
                                    dt.maxDepth, [3, 5, 7]).build()
regev_dt = RegressionEvaluator(labelCol='count(Date)')
# regev_dtk.evaluate(transformed_dtk.select("count(Date)", "prediction"), {regev_dtk.metricName: "mae"})
cv = CrossValidator(estimator=dt, estimatorParamMaps=params, evaluator=regev_dt)
cv_model = cv.fit(training_df.select('count(Date)', 'id'))
transformed_dt_cv = cv_model.transform(testing_df)
regev_dt.evaluate(transformed_dt_cv, {regev_dt.metricName: "mae"})

295.0

In [ ]:
training_df.rdd.max

In [ ]:
iso = IsotonicRegression(labelCol="count(Date)", featuresCol="id")
model_iso = iso.fit(training_df.select("count(Date)", "id"))
transformed_iso = model_iso.transform(testing_df)

In [21]:
rf = RandomForestRegressor(numTrees=3, maxDepth=5,labelCol="count(Date)", featuresCol="id")
model_rf = rf.fit(training_df.select("count(Date)", "id"))
transformed_rd = model_rf.transform(testing_df)
transformed_rd.show(10)

+----------+-----------+------+------------------+
|      Date|count(Date)|    id|        prediction|
+----------+-----------+------+------------------+
|2016-05-01|       2200|[12.0]|2135.3333333333335|
|2016-06-01|       2006|[13.0]|2070.6666666666665|
+----------+-----------+------+------------------+



In [23]:
grouped_doctor = reading_mod.groupby('Date', 'Doctor').agg({'Doctor': 'count'})
grouped_doctor_with_date = grouped_doctor.withColumn('Date', change_to_date_func(col('Date')))
window_row_doctor = Window().partitionBy('Doctor').orderBy('Date')
grouped_doctor_with_date_new = grouped_doctor_with_date.withColumn('id', row_number().over(window_row_doctor))

In [24]:
strindexer = StringIndexer(inputCol="Doctor", outputCol="Doctor_idx")
model_strindexer = strindexer.fit(grouped_doctor_with_date_new)
grouped_doctor_with_date_new_indexed = model_strindexer.transform(grouped_doctor_with_date_new)
assembler = VectorAssembler(inputCols=["id", "Doctor_idx"], outputCol="features")
grouped_doctor_with_date_new_1 = assembler.transform(grouped_doctor_with_date_new_indexed)

In [ ]:
training_df = grouped_doctor_with_date_new_1
testing_df = grouped_doctor_with_date_new_1.where(col('Date') > datetime(2016,4,2))

In [ ]:
lrd = LinearRegression(maxIter=1000, regParam=0.01, elasticNetParam=0.0, labelCol="count(Doctor)", featuresCol="features")
model_lrd = lrd.fit(training_df.select("features", "count(Doctor)"))
transformed_lrd = model_lrd.transform(testing_df)

In [ ]:
regev = RegressionEvaluator(labelCol="count(Doctor)")
regev.evaluate(transformed_lrd.select("count(Doctor)", "prediction"), {regev.metricName: "mae"})

In [ ]:
isod = IsotonicRegression(labelCol="count(Doctor)", featuresCol="features")
model_isod = isod.fit(training_df.select("features", "count(Doctor)"))
transformed_isod = model_isod.transform(testing_df)
transformed_isod.show(100)

In [ ]:
grouped_doctor_with_date_new_1.show(10)

In [23]:
tr = grouped_doctor_with_date_new_1.where(col('Doctor_idx') >= 0.0).where(col(
        'Doctor_idx') < 2.0)
te = grouped_doctor_with_date_new_1.where(col('Date') > datetime(2016,4,2)).where(col('Doctor_idx') >= 0.0
        ).where(col('Doctor_idx') < 2.0)
dtk = DecisionTreeRegressor(maxDepth=32, labelCol="count(Doctor)", featuresCol="features")
model_dtk = dtk.fit(tr)
transformed_dtk = model_dtk.transform(te)

IllegalArgumentException: u'requirement failed: DecisionTree requires maxBins (= 26) to be at least as large as the number of values in each categorical feature, but categorical feature 1 has 178 values. Considering remove this and other categorical features with a large number of values, or add more training examples.'

In [76]:
#max_val_doctor = grouped_doctor_with_date_new_1.select('Doctor_idx').rdd.max()[0]

training_doctor_df = grouped_doctor_with_date_new_1.where(col('Date') < datetime(2016,4,2))
testing_doctor_df = grouped_doctor_with_date_new_1.where(col('Date') > datetime(2016,4,2))


dtk = DecisionTreeRegressor(maxDepth=3, labelCol="count(Doctor)", featuresCol="features", maxBins=178)
model_dtk = dtk.fit(training_doctor_df)
transformed_dtk = model_dtk.transform(testing_doctor_df)
regev_dtk = RegressionEvaluator(labelCol="count(Doctor)")
regev_dtk.evaluate(transformed_dtk.select("count(Doctor)", "prediction"), {regev_dtk.metricName: "mae"})

8.921920050544317

In [31]:
transformed_dtk.show(100)

+----------+------------------+-------------+---+----------+-----------+------------------+
|      Date|            Doctor|count(Doctor)| id|Doctor_idx|   features|        prediction|
+----------+------------------+-------------+---+----------+-----------+------------------+
|2016-05-01|        Sam Slater|            4| 12|       9.0| [12.0,9.0]|3.0526315789473686|
|2016-06-01|        Sam Slater|            4| 13|       9.0| [13.0,9.0]|3.0526315789473686|
|2016-05-01| Alexander Skinner|           38| 12|      11.0|[12.0,11.0]|16.761904761904763|
|2016-06-01| Alexander Skinner|           32| 13|      11.0|[13.0,11.0]|16.761904761904763|
|2016-05-01|    Sally Morrison|           13| 11|      65.0|[11.0,65.0]|16.761904761904763|
|2016-06-01|    Sally Morrison|           22| 12|      65.0|[12.0,65.0]|16.761904761904763|
|2016-06-01|    Brandon Fraser|            1| 11|      85.0|[11.0,85.0]|3.0526315789473686|
|2016-05-01|       Joseph Ince|            1|  1|     177.0|[1.0,177.0]| 37.0571

In [34]:
rfd = RandomForestRegressor(numTrees=10, maxDepth=3,labelCol="count(Doctor)", featuresCol="features", maxBins=178)
model_rfd = rfd.fit(training_doctor_df.select("count(Doctor)", "features"))
transformed_rfd = model_rfd.transform(testing_doctor_df)
regev_rfd = RegressionEvaluator(labelCol="count(Doctor)")
regev_rfd.evaluate(transformed_rfd.select("count(Doctor)", "prediction"), {regev_rfd.metricName: "r2"})
#transformed_rfd.show(10)

0.0768890555258217

In [ ]:
grouped_specialty = reading_mod.groupby('Date', 'Specialty').agg({'Specialty': 'count'})
grouped_specialty_with_date = grouped_specialty.withColumn('Date', change_to_date_func(col('Date')))
window_row_specialty = Window().partitionBy('Specialty').orderBy('Date')
grouped_specialty_with_date_new = grouped_specialty_with_date.withColumn('id', 
                                                                         row_number().over(window_row_specialty))
strindexer = StringIndexer(inputCol="Specialty", outputCol="Specialty_idx")
model_strindexer = strindexer.fit(grouped_specialty_with_date_new)
grouped_specialty_with_date_new_indexed = model_strindexer.transform(grouped_specialty_with_date_new)
assembler = VectorAssembler(inputCols=["id", "Specialty_idx"], outputCol="features")
grouped_specialty_with_date_new_1 = assembler.transform(grouped_specialty_with_date_new_indexed)
training_specialty_df = grouped_specialty_with_date_new_1
testing_specialty_df = grouped_specialty_with_date_new_1.where(col('Date') > datetime(2016,4,2))
lrs = LinearRegression(maxIter=1000, regParam=0.01, elasticNetParam=0.0, labelCol="count(Specialty)", 
                       featuresCol="features")
model_lrs = lrs.fit(training_specialty_df.select("features", "count(Specialty)"))
transformed_lrs = model_lrs.transform(testing_specialty_df)
regev_s = RegressionEvaluator(labelCol="count(Specialty)")
regev_s.evaluate(transformed_lrs.select("count(Specialty)", "prediction"), {regev_s.metricName: "mae"})

In [ ]:
transformed_lrs.show(100)

In [ ]:
isos = IsotonicRegression(labelCol="count(Specialty)", featuresCol="features")
model_isos = isos.fit(training_specialty_df.select("features", "count(Specialty)"))
transformed_isos = model_isos.transform(testing_specialty_df)
regev_s = RegressionEvaluator(labelCol="count(Specialty)")
regev_s.evaluate(transformed_isos.select("count(Specialty)", "prediction"), {regev_s.metricName: "mae"})

In [ ]:
transformed_isos.show(100)

In [ ]:
dts = DecisionTreeRegressor(maxDepth=3, labelCol="count(Specialty)", featuresCol="features")
model_dts = dts.fit(training_specialty_df.select('count(Specialty)', 'features'))
transformed_dts = model_dts.transform(testing_specialty_df)
regev_dts = RegressionEvaluator(labelCol="count(Specialty)")
regev_dts.evaluate(transformed_dts.select("count(Specialty)", "prediction"), {regev_dts.metricName: "mae"})

In [ ]:
transformed_dts.show(100)

In [ ]:
grouped_hospital = reading_mod.groupby('Date', 'Hospital').agg({'Hospital': 'count'})
grouped_hospital_with_date = grouped_hospital.withColumn('Date', change_to_date_func(col('Date')))
window_row_hospital = Window().partitionBy('Hospital').orderBy('Date')
grouped_hospital_with_date_new = grouped_hospital_with_date.withColumn('id', 
                                                                         row_number().over(window_row_hospital))
strindexer = StringIndexer(inputCol="Hospital", outputCol="Hospital_idx")
model_strindexer = strindexer.fit(grouped_hospital_with_date_new)
grouped_hospital_with_date_new_indexed = model_strindexer.transform(grouped_hospital_with_date_new)
assembler = VectorAssembler(inputCols=["id", "Hospital_idx"], outputCol="features")
grouped_hospital_with_date_new_1 = assembler.transform(grouped_hospital_with_date_new_indexed)
training_hospital_df = grouped_hospital_with_date_new_1
testing_hospital_df = grouped_hospital_with_date_new_1.where(col('Date') > datetime(2016,4,2))
lrh = LinearRegression(maxIter=1000, regParam=0.0, elasticNetParam=1.0, labelCol="count(Hospital)", 
                       featuresCol="features")
model_lrh = lrh.fit(training_hospital_df.select("features", "count(Hospital)"))
transformed_lrh = model_lrh.transform(testing_hospital_df)
regev_h = RegressionEvaluator(labelCol="count(Hospital)")
regev_h.evaluate(transformed_lrh.select("count(Hospital)", "prediction"), {regev_s.metricName: "mae"})

In [ ]:
transformed_lrh.show(100)

In [ ]:
isoh = IsotonicRegression(labelCol="count(Hospital)", featuresCol="features")
model_isoh = isoh.fit(training_hospital_df.select("features", "count(Hospital)"))
transformed_isoh = model_isoh.transform(testing_hospital_df)
regev_h = RegressionEvaluator(labelCol="count(Hospital)")
regev_h.evaluate(transformed_isoh.select("count(Hospital)", "prediction"), {regev_s.metricName: "mae"})

In [ ]:
transformed_isoh.show(100)

In [ ]:
dth = DecisionTreeRegressor(maxDepth=3, labelCol="count(Hospital)", featuresCol="features")
model_dth = dth.fit(training_hospital_df.select('count(Hospital)', 'features'))
transformed_dth = model_dth.transform(testing_hospital_df)
regev_dth = RegressionEvaluator(labelCol="count(Hospital)")
regev_dth.evaluate(transformed_dth.select("count(Hospital)", "prediction"), {regev_dth.metricName: "mae"})

In [ ]:
transformed_dth.show(100)

In [44]:
grouped_proc = reading_mod.groupby('Date', 'Procedure').agg({'Procedure': 'count'})
grouped_proc_with_date = grouped_proc.withColumn('Date', change_to_date_func(col('Date')))
window_row_proc = Window().partitionBy('Procedure').orderBy('Date')
grouped_proc_with_date_new = grouped_proc_with_date.withColumn('id', row_number().over(window_row_proc))
strindexer = StringIndexer(inputCol="Procedure", outputCol="Procedure_idx")
model_strindexer = strindexer.fit(grouped_proc_with_date_new)
grouped_proc_with_date_new_indexed = model_strindexer.transform(grouped_proc_with_date_new)
assembler = VectorAssembler(inputCols=["id", "Procedure_idx"], outputCol="features")
grouped_proc_with_date_new_1 = assembler.transform(grouped_proc_with_date_new_indexed)
training_proc_df = grouped_proc_with_date_new_1
testing_proc_df = grouped_proc_with_date_new_1.where(col('Date') > datetime(2016,4,2))
dtproc = DecisionTreeRegressor(maxDepth=5, labelCol="count(Procedure)", featuresCol="features", maxBins=128)
model_dtproc = dtproc.fit(training_proc_df.select('count(Procedure)', 'features'))
transformed_dtproc = model_dtproc.transform(testing_proc_df)
regev_dtproc = RegressionEvaluator(labelCol="count(Procedure)")
regev_dtproc.evaluate(transformed_dtproc.select("count(Procedure)", "prediction"), {regev_dtproc.metricName: "mae"})

3.5804591435590853

In [45]:
transformed_dtproc.show(10)

+----------+--------------------+----------------+---+-------------+-----------+------------------+
|      Date|           Procedure|count(Procedure)| id|Procedure_idx|   features|        prediction|
+----------+--------------------+----------------+---+-------------+-----------+------------------+
|2016-05-01|Aortic Valve Surgery|               7| 11|         33.0|[11.0,33.0]|10.244444444444444|
|2016-06-01|Aortic Valve Surgery|               4| 12|         33.0|[12.0,33.0]|10.244444444444444|
|2016-05-01|Clipping Cerebral...|               8| 11|         52.0|[11.0,52.0]|10.244444444444444|
|2016-06-01|Clipping Cerebral...|               7| 12|         52.0|[12.0,52.0]|10.244444444444444|
|2016-05-01|   Other Orthopaedic|              14| 11|         72.0|[11.0,72.0]|20.517241379310345|
|2016-06-01|   Other Orthopaedic|              14| 12|         72.0|[12.0,72.0]|20.517241379310345|
|2016-05-01|    Ovary Operations|              10| 11|         34.0|[11.0,34.0]|10.244444444444444|


In [33]:
grouped_pr = reading_mod.groupby('Date', 'Priority').agg({'Priority': 'count'})
grouped_pr_with_date = grouped_pr.withColumn('Date', change_to_date_func(col('Date')))
window_row_pr = Window().partitionBy('Priority').orderBy('Date')
grouped_pr_with_date_new = grouped_pr_with_date.withColumn('id', row_number().over(window_row_pr))
strindexer = StringIndexer(inputCol="Priority", outputCol="Priority_idx")
model_strindexer = strindexer.fit(grouped_pr_with_date_new)
grouped_pr_with_date_new_indexed = model_strindexer.transform(grouped_pr_with_date_new)
assembler = VectorAssembler(inputCols=["id", "Priority_idx"], outputCol="features")
grouped_pr_with_date_new_1 = assembler.transform(grouped_pr_with_date_new_indexed)
training_pr_df = grouped_pr_with_date_new_1
testing_pr_df = grouped_pr_with_date_new_1.where(col('Date') > datetime(2016,4,2))
dtpr = DecisionTreeRegressor(maxDepth=3, labelCol="count(Priority)", featuresCol="features")
model_dtpr = dtpr.fit(training_pr_df.select('count(Priority)', 'features'))
transformed_dtpr = model_dtpr.transform(testing_pr_df)
regev_dtpr = RegressionEvaluator(labelCol="count(Priority)")
regev_dtpr.evaluate(transformed_dtpr.select("count(Priority)", "prediction"), {regev_dtpr.metricName: "r2"})

0.7193436290584654

In [32]:
transformed_dtpr.show(100)

+----------+--------+---------------+---+------------+----------+-----------------+
|      Date|Priority|count(Priority)| id|Priority_idx|  features|       prediction|
+----------+--------+---------------+---+------------+----------+-----------------+
|2016-05-01|       B|            872| 12|         1.0|[12.0,1.0]|832.6666666666666|
|2016-06-01|       B|            883| 13|         1.0|[13.0,1.0]|832.6666666666666|
|2016-05-01|       C|            732| 12|         2.0|[12.0,2.0]|            586.0|
|2016-06-01|       C|            621| 13|         2.0|[13.0,2.0]|            586.0|
|2016-05-01|       A|            596| 12|         0.0|[12.0,0.0]|            586.0|
|2016-06-01|       A|            502| 13|         0.0|[13.0,0.0]|            586.0|
+----------+--------+---------------+---+------------+----------+-----------------+

